# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Project Setup

In [59]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [136]:
# all import statements needed for the project, for example:

import os

import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import re
import geopandas as gpd
import math
import glob
import numpy as np
import sqlite3
import sqlalchemy as db
from sqlalchemy import create_engine

In [137]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TLC_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

PARQUET_FILES = "parquet_files"
TAXI_ZONES_DIR = "taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
WEATHER_CSV_DIR = "weather_data"

#CRS = 4326  # coordinate reference system

# (lat, lon)
#NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [138]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

### Download All data

In [25]:
def get_all_urls_from_tlc_page():
    response = requests.get(TLC_URL)
    html = response.content
    return html

In [26]:
#Identifies all of the yellow and fhvhv parquet files for years 2020 - 2024
pattern = re.compile(r".*(yellow|fhvhv).*(2020|2021|2022|2023|2024)-\d{2}\.parquet")

def filter_parquet_urls():
    html = get_all_urls_from_tlc_page()
    soup = bs4.BeautifulSoup(html, "html.parser")
    urls = soup.find_all("a", href=pattern)
    parquet_urls = [link["href"].strip() for link in urls]
    return parquet_urls

In [27]:
folder_name = "parquet_files"

# Check if the folder exists
if not os.path.exists(folder_name):
    os.mkdir(folder_name)
    print(f"Folder '{folder_name}' created successfully!")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'parquet_files' already exists.


In [28]:
# parses the filename from the link and then downloads the files one by one
def download_parquet_files():
    for link in filter_parquet_urls():
        filename = link.split("/")[-1]
        r = requests.get(link)
        with open(f"parquet_files/{filename}", "wb") as f:
            f.write(r.content)

#run the first time to download data
#download_parquet_files()

### Load Taxi Zones & Parquet Files

In [139]:
#Reads the shape file
def load_taxi_zones(shapefile):
    taxi_zones = gpd.read_file(shapefile)
    return taxi_zones

In [140]:
gdf_taxi_zones = load_taxi_zones(TAXI_ZONES_SHAPEFILE)
print(gdf_taxi_zones.head())

   OBJECTID  Shape_Leng  Shape_Area                     zone  LocationID  \
0         1    0.116357    0.000782           Newark Airport           1   
1         2    0.433470    0.004866              Jamaica Bay           2   
2         3    0.084341    0.000314  Allerton/Pelham Gardens           3   
3         4    0.043567    0.000112            Alphabet City           4   
4         5    0.092146    0.000498            Arden Heights           5   

         borough                                           geometry  
0            EWR  POLYGON ((9.33e+05 1.93e+05, 9.33e+05 1.93e+05...  
1         Queens  MULTIPOLYGON (((1.03e+06 1.72e+05, 1.03e+06 1....  
2          Bronx  POLYGON ((1.03e+06 2.57e+05, 1.03e+06 2.57e+05...  
3      Manhattan  POLYGON ((9.92e+05 2.04e+05, 9.92e+05 2.04e+05...  
4  Staten Island  POLYGON ((9.36e+05 1.44e+05, 9.36e+05 1.44e+05...  


In [141]:
# converts taxi zone geometry coordinates to the appropriate coordinate system  
gdf_taxi_zones = gdf_taxi_zones.to_crs(epsg=4326)
gdf_taxi_zones

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.184 40.695, -74.184 40.695, -74...."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.823 40.639, -73.823 40.636..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.848 40.871, -73.847 40.871, -73...."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.972 40.726, -73.972 40.726, -73...."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.174 40.563, -74.173 40.562, -74...."
...,...,...,...,...,...,...,...
258,259,0.126750,0.000395,Woodlawn/Wakefield,259,Bronx,"POLYGON ((-73.851 40.91, -73.852 40.909, -73.8..."
259,260,0.133514,0.000422,Woodside,260,Queens,"POLYGON ((-73.902 40.761, -73.901 40.76, -73.9..."
260,261,0.027120,0.000034,World Trade Center,261,Manhattan,"POLYGON ((-74.013 40.705, -74.013 40.705, -74...."
261,262,0.049064,0.000122,Yorkville East,262,Manhattan,"MULTIPOLYGON (((-73.944 40.783, -73.944 40.783..."


In [142]:
# load parquet file into a pandas DataFrame
def load_parquet_file(file_path):
    df = pd.read_parquet(file_path)
    return df

In [143]:
# load a random yellow taxi trip parquet file to check if the function works correctly for testing purposes
example = os.path.join(PARQUET_FILES, "yellow_tripdata_2023-01.parquet")
example_df = load_parquet_file(example)

# preview the data
print(example_df.head())

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         2  2023-01-01 00:32:10   2023-01-01 00:40:36              1.0   
1         2  2023-01-01 00:55:08   2023-01-01 01:01:27              1.0   
2         2  2023-01-01 00:25:04   2023-01-01 00:37:49              1.0   
3         1  2023-01-01 00:03:48   2023-01-01 00:13:25              0.0   
4         2  2023-01-01 00:10:29   2023-01-01 00:21:19              1.0   

   trip_distance  RatecodeID store_and_fwd_flag  PULocationID  DOLocationID  \
0           0.97         1.0                  N           161           141   
1           1.10         1.0                  N            43           237   
2           2.51         1.0                  N            48           238   
3           1.90         1.0                  N           138             7   
4           1.43         1.0                  N           107            79   

   payment_type  fare_amount  extra  mta_tax  tip_amount  tolls_amount  \


In [144]:
# load a random High-Volume For-Hire Vehicle trip parquet file to check if the function works correctly
example2 = os.path.join(PARQUET_FILES, "fhvhv_tripdata_2023-01.parquet")
example_df2 = load_parquet_file(example2)

# preview the data
print(example_df2.head())

  hvfhs_license_num dispatching_base_num originating_base_num  \
0            HV0003               B03404               B03404   
1            HV0003               B03404               B03404   
2            HV0003               B03404               B03404   
3            HV0003               B03404               B03404   
4            HV0003               B03404               B03404   

     request_datetime   on_scene_datetime     pickup_datetime  \
0 2023-01-01 00:18:06 2023-01-01 00:19:24 2023-01-01 00:19:38   
1 2023-01-01 00:48:42 2023-01-01 00:56:20 2023-01-01 00:58:39   
2 2023-01-01 00:15:35 2023-01-01 00:20:14 2023-01-01 00:20:27   
3 2023-01-01 00:35:24 2023-01-01 00:39:30 2023-01-01 00:41:05   
4 2023-01-01 00:43:15 2023-01-01 00:51:10 2023-01-01 00:52:47   

     dropoff_datetime  PULocationID  DOLocationID  trip_miles  ...  sales_tax  \
0 2023-01-01 00:48:07            48            68        0.94  ...       2.30   
1 2023-01-01 01:33:08           246           163       

In [145]:
example_df2.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag'],
      dtype='object')

In [146]:
example_df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

### Cleaning and Filtering
* Remove all non-Uber data from fhvhv
* Remove all invalid pickup and dropoff location IDs for both uber and yellow taxi, where ID is greater than 263 using the `shp` file
* Remove unnecessary columns and only keeping columns needed to answer questions in the other parts of this project
* Remove invalid data points (use your discretion!)
* normalize column names; 
normalieg and using appropriate column types for the respective dat

* Remove trips from both uber and yellow taxi that start and/or end outside of the following latitude/longitude coordinate box: (40.560445, -74.242330) and (40.908524, -73.71704).


In [147]:
#Compute the center of the taxi zones for easier comparison and adds a column to the df of our shapefile 
gdf_taxi_zones['centroid'] = gdf_taxi_zones.geometry.centroid

#Rename 'LocationID' to 'location_id' for consistency
gdf_taxi_zones = gdf_taxi_zones.rename(columns={'LocationID': 'location_id'})

#Removes the bulky geometry column after using it to compute centroid. 
gdf_taxi_zones = gdf_taxi_zones[['zone', 'location_id', 'centroid']]

/var/folders/wc/z6g_m0_n659dtcmkkgf76pv40000gn/T/ipykernel_30577/103723099.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_taxi_zones['centroid'] = gdf_taxi_zones.geometry.centroid


In [148]:
gdf_taxi_zones

,zone,location_id,centroid
0,Newark Airport,1,POINT (-74.174 40.692)
1,Jamaica Bay,2,POINT (-73.831 40.617)
2,Allerton/Pelham Gardens,3,POINT (-73.847 40.864)
3,Alphabet City,4,POINT (-73.977 40.724)
4,Arden Heights,5,POINT (-74.188 40.553)
...,...,...,...
258,Woodlawn/Wakefield,259,POINT (-73.852 40.898)
259,Woodside,260,POINT (-73.906 40.744)
260,World Trade Center,261,POINT (-74.013 40.709)
261,Yorkville East,262,POINT (-73.947 40.776)


### Calculate Sample Size

In [149]:
# default: 95% confidence interval, 5% margin of error, p of 0.5 (estimated) proportion of the population which has the attribute in question
def cochran_sample_size(population_size):
    z_score=1.96
    margin_of_error=0.05
    p=0.5
    sample_size = ((z_score**2)*p*(1-p)) / (margin_of_error**2)
    adjusted_sample_size = sample_size / (1 + ((sample_size-1)/population_size))

    return int(adjusted_sample_size)

### Common Functions

In [150]:
#Filter function to remove unecessary rows
def filter_data(data):
    #Ensure PU and DO locations are within valid location IDs (<= 263)
    data = data[(data['pickup_location_id'] <= 263) & (data['dropoff_location_id'] <= 263)]
    #Filters out rides where pickup and dropoff locations are the same
    filtered_data = data[data['trip_distance'] != 0]
        
    return filtered_data

In [151]:
# Removes trips from both uber and yellow taxi that start and/or end outside of the following latitude/longitude coordinate box:
def find_centroid(data):
    LAT_MIN, LON_MIN = 40.560445, -74.242330
    LAT_MAX, LON_MAX = 40.908524, -73.717047
    
    # Extract latitude and longitude from the 'centroid' column using .apply()
    data['centroid_lat'] = data['centroid'].apply(lambda point: point.y)
    data['centroid_lon'] = data['centroid'].apply(lambda point: point.x)
    
    # Filter rows where the centroid coordinates are within the bounding coordinate box
    centroid_data = data[
        (data['centroid_lat'] >= LAT_MIN) & (data['centroid_lat'] <= LAT_MAX) &
        (data['centroid_lon'] >= LON_MIN) & (data['centroid_lon'] <= LON_MAX)
    ]
    return centroid_data

### Process Taxi Data

In [152]:
#Grab all of the parquet files in the directory. glob.glob is used to identify/match the pattern, path.join retrieves all the paths 
all_taxi_parquet_files = glob.glob(os.path.join(PARQUET_FILES, "*yellow*.parquet"))

In [153]:
taxi_columns_mapping = {
    'tpep_pickup_datetime': 'pickup_datetime',
    'tpep_dropoff_datetime': 'dropoff_datetime',
    'extra': 'rush_hour_surcharge',
    'PULocationID': 'pickup_location_id',
    'DOLocationID': 'dropoff_location_id',
     'fare_amount': 'base_passenger_fare',
    'RatecodeID': 'rate_code_id'
}

#Make a list of just the columns we need for analysis
columns_to_keep = [
    'pickup_datetime', 'dropoff_datetime', 'trip_distance', 'rate_code_id',
    'pickup_location_id', 'dropoff_location_id', 'base_passenger_fare', 'rush_hour_surcharge', 'mta_tax',
    'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount',
    'congestion_surcharge', 'airport_fee'
]



#Create samples of all taxi parquet files according to cochran's sample size formula. Later, we concatenate all sample dfs into one df. 
sampled_taxi_dfs = []

for file_path in all_taxi_parquet_files:      
    taxi_df = load_parquet_file(file_path) #Makes a df for every parquet file 
    taxi_df = taxi_df.rename(columns=taxi_columns_mapping)
    
    population_size = len(taxi_df)
    sample_size = cochran_sample_size(population_size)
    sampled_taxi_df = taxi_df.sample(n=sample_size, random_state=42)
    #We found that there were a few files that did not have airport_fee as a column. We populate airport_fee with NaN for such parquet files.
    for col in columns_to_keep:  
        if col not in sampled_taxi_df.columns:
            sampled_taxi_df[col] = np.nan 
    sampled_taxi_df = sampled_taxi_df[columns_to_keep]
    sampled_taxi_dfs.append(sampled_taxi_df)

    # create one gigantic dataframe with data from every month needed
sampled_taxi_data = pd.concat(sampled_taxi_dfs)

sampled_taxi_data = filter_data(sampled_taxi_data)

# Make a single df that includes the taxi rides and their corresponding coordinates by merging the shape file with the ride files.
final_taxi_data = pd.merge(sampled_taxi_data, gdf_taxi_zones, left_on = 'pickup_location_id', right_on = 'location_id', how="inner")

final_taxi_data = find_centroid(final_taxi_data)

#Drop the centroid column, and just keep the latitude and longitude columns as that is more compatible for SQL 
final_taxi_data = final_taxi_data.drop(columns=['centroid'])

/var/folders/wc/z6g_m0_n659dtcmkkgf76pv40000gn/T/ipykernel_30577/4192337244.py:39: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sampled_taxi_data = pd.concat(sampled_taxi_dfs)


In [186]:
final_taxi_data.head()

,pickup_datetime,dropoff_datetime,trip_distance,rate_code_id,pickup_location_id,dropoff_location_id,base_passenger_fare,rush_hour_surcharge,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,zone,location_id,centroid_lat,centroid_lon
0,2023-06-28 08:27:09,2023-06-28 09:21:52,10.22,1.0,138,144,57.6,5.0,0.5,17.09,0.00,1.0,85.44,2.5,NaN,LaGuardia Airport,138,40.774375,-73.873629
1,2023-06-13 22:05:38,2023-06-13 22:10:48,0.80,1.0,263,237,7.2,3.5,0.5,0.00,0.00,1.0,12.20,2.5,NaN,Yorkville West,263,40.778765,-73.951010
2,2023-06-09 10:25:49,2023-06-09 10:41:51,1.62,1.0,162,236,15.6,0.0,0.5,2.00,0.00,1.0,21.60,2.5,NaN,Midtown East,162,40.756687,-73.972356
3,2023-06-28 15:56:14,2023-06-28 17:22:03,18.90,3.0,229,1,114.7,0.0,0.0,27.65,22.75,1.0,166.10,0.0,NaN,Sutton Place/Turtle Bay North,229,40.756728,-73.965146
4,2023-06-22 07:12:42,2023-06-22 07:23:47,1.52,1.0,43,74,11.4,0.0,0.5,1.94,0.00,1.0,14.84,0.0,NaN,Central Park,43,40.782478,-73.965553


In [155]:
final_taxi_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20826 entries, 0 to 20828
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   pickup_datetime        20826 non-null  datetime64[us]
 1   dropoff_datetime       20826 non-null  datetime64[us]
 2   trip_distance          20826 non-null  float64       
 3   rate_code_id           19770 non-null  float64       
 4   pickup_location_id     20826 non-null  int64         
 5   dropoff_location_id    20826 non-null  int64         
 6   base_passenger_fare    20826 non-null  float64       
 7   rush_hour_surcharge    20826 non-null  float64       
 8   mta_tax                20826 non-null  float64       
 9   tip_amount             20826 non-null  float64       
 10  tolls_amount           20826 non-null  float64       
 11  improvement_surcharge  20826 non-null  float64       
 12  total_amount           20826 non-null  float64       
 13  conges

In [156]:
final_taxi_data.describe()

,pickup_datetime,dropoff_datetime,trip_distance,rate_code_id,pickup_location_id,dropoff_location_id,base_passenger_fare,rush_hour_surcharge,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,location_id,centroid_lat,centroid_lon
count,20826,20826,20826.000000,19770.000000,20826.000000,20826.000000,20826.000000,20826.000000,20826.000000,20826.000000,20826.000000,20826.000000,20826.000000,19770.000000,7947.000000,20826.000000,20826.000000,20826.000000
mean,2022-05-01 10:17:07.596273,2022-05-01 10:33:09.629309,3.275384,1.415023,164.151157,160.661001,15.153432,1.205869,0.490733,2.680401,0.431877,0.543945,22.188515,2.297547,0.086825,164.151157,40.753285,-73.966859
min,2020-01-01 00:11:06,2020-01-01 00:30:50,0.010000,1.000000,4.000000,1.000000,-81.520000,-7.500000,-0.500000,0.000000,-34.200000,-1.000000,-108.200000,-2.500000,-1.250000,4.000000,40.576961,-74.029892
25%,2021-02-28 21:30:14.250000,2021-02-28 21:36:58.750000,1.090000,1.000000,132.000000,107.000000,7.200000,0.000000,0.500000,0.000000,0.000000,0.300000,12.600000,2.500000,0.000000,132.000000,40.740337,-73.989844
50%,2022-04-30 15:36:52.500000,2022-04-30 16:06:25,1.800000,1.000000,162.000000,161.000000,10.700000,0.500000,0.500000,2.160000,0.000000,0.300000,16.800000,2.500000,0.000000,162.000000,40.758027,-73.977698
75%,2023-06-29 14:40:03,2023-06-29 14:50:15.250000,3.310000,1.000000,234.000000,234.000000,17.000000,2.500000,0.500000,3.440000,0.000000,1.000000,24.360000,2.500000,0.000000,234.000000,40.773633,-73.961763
max,2024-08-31 22:43:47,2024-08-31 23:26:23,67.900000,99.000000,263.000000,263.000000,209.500000,11.750000,0.500000,50.000000,40.000000,1.000000,262.700000,2.500000,1.250000,263.000000,40.899529,-73.739337
std,NaN,NaN,4.121171,6.028092,65.215425,70.490221,13.948344,1.512793,0.089069,3.157641,1.825400,0.351174,17.759500,0.735558,0.321505,65.215425,0.032466,0.045223


### Processing Uber Data

In [157]:
#Grab all of the parquet files in the directory. glob.glob is used to identify/match the pattern, path.join retrieves all the paths 
all_fhvhv_parquet_files = glob.glob(os.path.join(PARQUET_FILES, "*fhvhv*.parquet"))

In [158]:
uber_columns_mapping = {
    'trip_miles': 'trip_distance',
    'PULocationID': 'pickup_location_id',
    'DOLocationID': 'dropoff_location_id',
    'tolls': 'tolls_amount',
    'tips': 'tip_amount',
    'bcf': 'black_car_fund_fee',
    
}

#Create samples of all uber parquet files according to cochran's sample size formula. Later, we concatenate all sample dfs into one df. 
sampled_uber_dfs = []
columns_to_keep = ['hvfhs_license_num',
       'request_datetime', 'pickup_datetime',
       'dropoff_datetime', 'pickup_location_id', 'dropoff_location_id', 'trip_distance',
        'base_passenger_fare', 'tolls_amount', 'black_car_fund_fee', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tip_amount']

for file_path in all_fhvhv_parquet_files:      
    uber_df = load_parquet_file(file_path) #Makes a df for every parquet file 
    uber_df = uber_df.rename(columns=uber_columns_mapping)
    uber_df = uber_df[uber_df['hvfhs_license_num'] == 'HV0003'] #Filters out non-uber rides from the hvfhs files before creating samples
    population_size = len(uber_df)
    sample_size = cochran_sample_size(population_size)
    sampled_uber_df = uber_df.sample(n=sample_size, random_state=42)
    sampled_uber_df = sampled_uber_df[columns_to_keep]
    sampled_uber_dfs.append(sampled_uber_df)

    # create one gigantic dataframe with data from every month needed
sampled_uber_data = pd.concat(sampled_uber_dfs)

sampled_uber_data = filter_data(sampled_uber_data)

# Make a single df that includes the taxi rides and their corresponding coordinates by merging the shape file with the ride files.
final_uber_data = pd.merge(sampled_uber_data, gdf_taxi_zones, left_on = 'pickup_location_id', right_on = 'location_id', how="inner")

final_uber_data = find_centroid(final_uber_data)

#Remove the centroid column and just keep the latitude and longitude for SQL compatibility
final_uber_data = final_uber_data.drop(columns=['centroid'])

/var/folders/wc/z6g_m0_n659dtcmkkgf76pv40000gn/T/ipykernel_30577/3122417029.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sampled_uber_data = pd.concat(sampled_uber_dfs)


In [189]:
final_uber_data.head()

,hvfhs_license_num,request_datetime,pickup_datetime,dropoff_datetime,pickup_location_id,dropoff_location_id,trip_distance,base_passenger_fare,tolls_amount,black_car_fund_fee,sales_tax,congestion_surcharge,airport_fee,tip_amount,zone,location_id,centroid_lat,centroid_lon
0,HV0003,2021-03-19 08:07:07,2021-03-19 08:11:50,2021-03-19 08:20:59,97,231,2.52,12.58,0.00,0.39,1.14,2.75,NaN,0.0,Fort Greene,97,40.690786,-73.974882
1,HV0003,2021-03-01 22:02:07,2021-03-01 22:08:28,2021-03-01 22:20:19,247,20,2.41,11.94,0.00,0.36,1.06,0.00,NaN,0.0,West Concourse,247,40.828988,-73.924409
2,HV0003,2021-03-13 05:42:59,2021-03-13 05:48:40,2021-03-13 05:55:36,18,247,2.21,15.25,0.00,0.46,1.35,0.00,NaN,0.0,Bedford Park,18,40.867682,-73.890183
3,HV0003,2021-03-12 20:58:30,2021-03-12 21:02:02,2021-03-12 21:25:13,211,229,3.85,21.51,0.00,0.65,1.91,2.75,NaN,0.0,SoHo,211,40.723888,-74.001537
4,HV0003,2021-03-31 08:42:41,2021-03-31 08:49:53,2021-03-31 09:56:02,74,45,18.96,71.31,6.12,2.33,6.89,2.75,0.0,0.0,East Harlem North,74,40.801169,-73.937345


In [160]:
final_uber_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20678 entries, 0 to 20731
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hvfhs_license_num     20678 non-null  object        
 1   request_datetime      20678 non-null  datetime64[us]
 2   pickup_datetime       20678 non-null  datetime64[us]
 3   dropoff_datetime      20678 non-null  datetime64[us]
 4   pickup_location_id    20678 non-null  int64         
 5   dropoff_location_id   20678 non-null  int64         
 6   trip_distance         20678 non-null  float64       
 7   base_passenger_fare   20678 non-null  float64       
 8   tolls_amount          20678 non-null  float64       
 9   black_car_fund_fee    20678 non-null  float64       
 10  sales_tax             20678 non-null  float64       
 11  congestion_surcharge  20678 non-null  float64       
 12  airport_fee           15107 non-null  float64       
 13  tip_amount           

In [161]:
final_uber_data.describe()

,request_datetime,pickup_datetime,dropoff_datetime,pickup_location_id,dropoff_location_id,trip_distance,base_passenger_fare,tolls_amount,black_car_fund_fee,sales_tax,congestion_surcharge,airport_fee,tip_amount,location_id,centroid_lat,centroid_lon
count,20678,20678,20678,20678.000000,20678.000000,20678.000000,20678.000000,20678.000000,20678.000000,20678.000000,20678.000000,15107.000000,20678.000000,20678.000000,20678.000000,20678.000000
mean,2022-04-29 03:51:06.178257,2022-04-29 03:55:45.584002,2022-04-29 04:13:38.630138,137.800077,137.759745,4.439277,21.201442,0.656449,0.620884,1.895659,1.052326,0.191468,0.820059,137.800077,40.737433,-73.934434
min,2020-01-01 01:14:43,2020-01-01 01:21:23,2020-01-01 01:46:38,3.000000,1.000000,0.020000,-9.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,40.561994,-74.170887
25%,2021-02-24 10:25:41.250000,2021-02-24 10:29:33.250000,2021-02-24 10:49:42.250000,73.000000,74.000000,1.540000,10.490000,0.000000,0.290000,0.910000,0.000000,0.000000,0.000000,73.000000,40.690786,-73.984196
50%,2022-04-25 21:30:18,2022-04-25 21:33:31.500000,2022-04-25 21:42:09,139.000000,138.000000,2.810000,16.720000,0.000000,0.470000,1.470000,0.000000,0.000000,0.000000,139.000000,40.737698,-73.948789
75%,2023-06-30 11:11:37,2023-06-30 11:14:15,2023-06-30 11:23:13.250000,211.000000,209.000000,5.710000,26.570000,0.000000,0.770000,2.380000,2.750000,0.000000,0.000000,211.000000,40.774375,-73.898956
max,2024-08-31 23:43:36,2024-08-31 23:49:13,2024-09-01 00:10:26,263.000000,263.000000,42.660000,255.840000,43.910000,7.950000,23.510000,2.750000,5.000000,40.000000,263.000000,40.899529,-73.726656
std,NaN,NaN,NaN,75.489021,75.367212,4.359484,15.753046,2.520313,0.500625,1.450202,1.332477,0.670128,2.429705,75.489021,0.069218,0.065377


### Processing Weather Data

In [162]:
def get_all_weather_csvs(directory):
    weather_dfs = []

    # Iterate over all files in the given directory
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path, low_memory=False)
        # Append the DataFrame to the list
        weather_dfs.append(df)
    weather_dfs = pd.concat(weather_dfs, ignore_index=True)
    return weather_dfs


In [163]:
columns_to_keep = ['DATE','LATITUDE', 'LONGITUDE', 'MonthlyTotalLiquidPrecipitation', 'DailyPrecipitation', 'HourlyPrecipitation', 'DailyAverageWindSpeed', 'HourlyWindSpeed', 'DailySnowfall']

weather_data = get_all_weather_csvs(WEATHER_CSV_DIR)
weather_data = weather_data[columns_to_keep]
weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])

# weather_data[weather_data["DailyPrecipitation"].isna()] #54343
weather_data[weather_data["HourlyPrecipitation"].isna()] #8,523
# weather_data


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 580: invalid start byte

In [164]:
#We needed to do this because an error was being thrown that the hourly precipitation values were strings. 
# Identify rows where HourlyPrecipitation is a string
string_precipitation_rows = weather_data[weather_data['HourlyPrecipitation'].apply(lambda x: isinstance(x, str))]

# Print or display the rows to inspect the problematic values
print(string_precipitation_rows)
# Identify rows where HourlyPrecipitation is a string
string_precipitation_rows = weather_data[weather_data['HourlyPrecipitation'].apply(lambda x: isinstance(x, str))]

# Extract the HourlyPrecipitation values that are strings
string_values = string_precipitation_rows['HourlyPrecipitation']

# Get unique values and their count using value_counts()
string_value_counts = string_values.value_counts()

# Print or display the unique string values and their count
print("Unique String Values and Their Counts in HourlyPrecipitation:")
print(string_value_counts)


Empty DataFrame
Columns: [DATE, LATITUDE, LONGITUDE, MonthlyTotalLiquidPrecipitation, DailyPrecipitation, HourlyPrecipitation, DailyAverageWindSpeed, HourlyWindSpeed, DailySnowfall, DATE_ONLY]
Index: []
Unique String Values and Their Counts in HourlyPrecipitation:
Series([], Name: count, dtype: int64)


In [165]:
#Cleans out the string values based on different cases identified in the query above
# Replace 'T' (trace amount) with 0.00
weather_data['HourlyPrecipitation'].replace('T', 0.00, inplace=True)

# Function to clean non-numeric characters from values
def clean_precipitation_value(value):
    if isinstance(value, str):
        # Remove all non-numeric characters except the decimal point
        return re.sub(r'[^0-9.]', '', value)
    return value

# Apply the cleaning function to the HourlyPrecipitation column
weather_data['HourlyPrecipitation'] = weather_data['HourlyPrecipitation'].apply(clean_precipitation_value)

# Convert all cleaned values to numeric, coercing any problematic ones to NaN
weather_data['HourlyPrecipitation'] = pd.to_numeric(weather_data['HourlyPrecipitation'], errors='coerce')

# Display the cleaned DataFrame
print(weather_data.head())

                 DATE  LATITUDE  LONGITUDE MonthlyTotalLiquidPrecipitation  \
0 2020-01-01 00:51:00  40.77898  -73.96925                             NaN   
1 2020-01-01 01:51:00  40.77898  -73.96925                             NaN   
2 2020-01-01 02:51:00  40.77898  -73.96925                             NaN   
3 2020-01-01 03:51:00  40.77898  -73.96925                             NaN   
4 2020-01-01 04:51:00  40.77898  -73.96925                             NaN   

  DailyPrecipitation  HourlyPrecipitation  DailyAverageWindSpeed  \
0                NaN                  0.0                    NaN   
1                NaN                  0.0                    NaN   
2                NaN                  0.0                    NaN   
3                NaN                  0.0                    NaN   
4                NaN                  0.0                    NaN   

   HourlyWindSpeed DailySnowfall   DATE_ONLY  
0              8.0           NaN  2020-01-01  
1              8.0          

/var/folders/wc/z6g_m0_n659dtcmkkgf76pv40000gn/T/ipykernel_30577/2630901844.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  weather_data['HourlyPrecipitation'].replace('T', 0.00, inplace=True)


In [166]:
# Fill in the remaining null hourly precipiation values 
# Ensure that 'DATE' is in the correct datetime format
weather_data['DATE'] = pd.to_datetime(weather_data['DATE'])

# Add a column for just the date (without time), which will help with grouping
weather_data['DATE_ONLY'] = weather_data['DATE'].dt.date
# Group by each unique date
daily_groups = weather_data.groupby('DATE_ONLY')

# Loop over each group of a specific date and calculate the remaining precipitation to distribute
for date, group in daily_groups:
    # Get the daily precipitation for the current day
    daily_precipitation = group['DailyPrecipitation'].iloc[0]
    
    # Calculate the sum of existing hourly precipitation values (if any)
    existing_hourly_precip = group['HourlyPrecipitation'].sum(skipna=True)

    # Calculate the remaining precipitation that needs to be distributed to missing hours
    remaining_precip = daily_precipitation - existing_hourly_precip

    # Get the number of hours with missing HourlyPrecipitation
    missing_hours = group['HourlyPrecipitation'].isna().sum()

    # Calculate how much to distribute to each missing hour
    if missing_hours > 0 and remaining_precip > 0:
        hourly_precipitation_to_assign = remaining_precip / missing_hours
    else:
        hourly_precipitation_to_assign = 0

    # Fill missing HourlyPrecipitation values with the calculated amount
    weather_data.loc[group.index, 'HourlyPrecipitation'] = group['HourlyPrecipitation'].fillna(hourly_precipitation_to_assign)


In [167]:
#Confirm there are no null hourly precipitation values anymore 
weather_data[weather_data["HourlyPrecipitation"].isna()] #0 

,DATE,LATITUDE,LONGITUDE,MonthlyTotalLiquidPrecipitation,DailyPrecipitation,HourlyPrecipitation,DailyAverageWindSpeed,HourlyWindSpeed,DailySnowfall,DATE_ONLY


In [191]:
weather_data.head()

,date,latitude,longitude,MonthlyTotalLiquidPrecipitation,daily_precipitation,hourly_precipitation,daily_average_wind_speed,hourly_wind_speed,daily_snowfall,DATE_ONLY
0,2020-01-01 00:51:00,40.77898,-73.96925,NaN,NaN,0.0,NaN,8.0,NaN,2020-01-01
1,2020-01-01 01:51:00,40.77898,-73.96925,NaN,NaN,0.0,NaN,8.0,NaN,2020-01-01
2,2020-01-01 02:51:00,40.77898,-73.96925,NaN,NaN,0.0,NaN,14.0,NaN,2020-01-01
3,2020-01-01 03:51:00,40.77898,-73.96925,NaN,NaN,0.0,NaN,11.0,NaN,2020-01-01
4,2020-01-01 04:51:00,40.77898,-73.96925,NaN,NaN,0.0,NaN,6.0,NaN,2020-01-01


In [169]:
weather_columns_mapping = {
 'DATE': 'date',
 'LATITUDE': 'latitude',
 'LONGITUDE': 'longitude',
 'DailyPrecipitation': 'daily_precipitation',
 'HourlyPrecipitation': 'hourly_precipitation',
 'DailyAverageWindSpeed': 'daily_average_wind_speed',
 'HourlyWindSpeed': 'hourly_wind_speed',
 'DailySnowfall': 'daily_snowfall'
}
weather_data.rename(columns=weather_columns_mapping, inplace=True)

In [192]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56098 entries, 0 to 56097
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   date                             56098 non-null  datetime64[ns]
 1   latitude                         56098 non-null  float64       
 2   longitude                        56098 non-null  float64       
 3   MonthlyTotalLiquidPrecipitation  52 non-null     object        
 4   daily_precipitation              1755 non-null   object        
 5   hourly_precipitation             56098 non-null  float64       
 6   daily_average_wind_speed         1697 non-null   float64       
 7   hourly_wind_speed                49660 non-null  float64       
 8   daily_snowfall                   1750 non-null   object        
 9   DATE_ONLY                        56098 non-null  object        
dtypes: datetime64[ns](1), float64(5), object(4)
memory usage: 

In [171]:
weather_data.describe()

,date,latitude,longitude,hourly_precipitation,daily_average_wind_speed,hourly_wind_speed
count,56098,5.609800e+04,5.609800e+04,56098.000000,1697.000000,49660.000000
mean,2022-05-29 21:14:19.618881024,4.077898e+01,-7.396925e+01,0.010511,5.000766,5.125453
min,2020-01-01 00:51:00,4.077898e+01,-7.396925e+01,0.000000,0.600000,0.000000
25%,2021-03-18 19:01:45,4.077898e+01,-7.396925e+01,0.000000,3.200000,3.000000
50%,2022-05-28 01:21:00,4.077898e+01,-7.396925e+01,0.000000,4.600000,5.000000
75%,2023-08-15 05:39:00,4.077898e+01,-7.396925e+01,0.000000,6.300000,7.000000
max,2024-10-22 18:51:00,4.077898e+01,-7.396925e+01,3.470000,14.200000,2237.000000
std,NaN,4.214267e-11,5.815134e-11,0.056783,2.339258,14.653212


In [195]:
# Create daily_weather DataFrame
daily_weather_data = weather_data[[
    'date', 'latitude', 'longitude', 'daily_precipitation', 
    'daily_average_wind_speed', 'daily_snowfall'
]].rename(columns={
    'daily_average_wind_speed': 'daily_wind_speed'
})


In [196]:
# Create hourly_weather DataFrame
hourly_weather_data = weather_data[[
    'date', 'latitude', 'longitude', 'hourly_precipitation', 
    'hourly_wind_speed', 'daily_snowfall'
]]


## Part 2: Storing Cleaned Data

In [172]:
engine = db.create_engine(DATABASE_URL)

In [173]:
import sqlite3

# this is our in-memory database, not stored on your hard drive
connection = sqlite3.connect(":memory:")

In [175]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATETIME,
    latitude REAL,
    longitude REAL,
    daily_precipitation REAL,
    daily_wind_speed REAL,
    daily_snowfall REAL
);
"""

HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATETIME,
    latitude REAL,
    longitude REAL,
    hourly_precipitation REAL,
    hourly_wind_speed REAL,
    daily_snowfall REAL,
    daily_id INTEGER, 
    FOREIGN KEY (daily_id) REFERENCES daily_weather(id)
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    trip_distance REAL,
    rate_code_id INTEGER,
    pickup_location_id INTEGER,
    dropoff_location_id INTEGER,
    base_passenger_fare REAL,
    rush_hour_surcharge REAL,
    mta_tax REAL,
    tip_amount REAL,
    tolls_amount REAL,
    improvement_surcharge REAL,
    total_amount REAL,
    congestion_surcharge REAL,
    airport_fee REAL
);
"""

# might not need request_datetime
UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    request_datetime DATETIME,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    trip_distance REAL,
    pickup_location_id INTEGER,
    dropoff_location_id INTEGER,
    base_passenger_fare REAL,
    tip_amount REAL,
    tolls_amount REAL,
    black_car_fund_fee REAL,
    total_amount REAL,
    sales_tax REAL,
    congestion_surcharge REAL,
    airport_fee REAL
);
"""


In [176]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [177]:
# first create a connection - we'll create a new database 
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///4501_Project.db", echo=True)

In [178]:
# create the tables with the schema files
with engine.connect() as connection:
    with open(DATABASE_SCHEMA_FILE, 'r') as schema_file:
        schema = schema_file.read()
        connection.execute(db.text(schema))

2024-12-02 19:24:30,824 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-02 19:24:30,838 INFO sqlalchemy.engine.Engine 
CREATE TABLE IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATETIME,
    latitude REAL,
    longitude REAL,
    hourly_precipitation REAL,
    hourly_wind_speed REAL,
    daily_snowfall REAL,
    daily_id INTEGER, 
    FOREIGN KEY (daily_id) REFERENCES daily_weather(id)
);

CREATE TABLE IF NOT EXISTS daily_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATETIME,
    latitude REAL,
    longitude REAL,
    daily_precipitation REAL,
    daily_wind_speed REAL,
    daily_snowfall REAL
);

CREATE TABLE IF NOT EXISTS taxi_trips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    trip_distance REAL,
    rate_code_id INTEGER,
    pickup_location_id INTEGER,
    dropoff_location_id INTEGER,
    base_passenger_fare REAL,
    rush_hour_surcharge REAL,
    mta_tax REAL,

ProgrammingError: (sqlite3.ProgrammingError) You can only execute one statement at a time.
[SQL: 
CREATE TABLE IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATETIME,
    latitude REAL,
    longitude REAL,
    hourly_precipitation REAL,
    hourly_wind_speed REAL,
    daily_snowfall REAL,
    daily_id INTEGER, 
    FOREIGN KEY (daily_id) REFERENCES daily_weather(id)
);

CREATE TABLE IF NOT EXISTS daily_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATETIME,
    latitude REAL,
    longitude REAL,
    daily_precipitation REAL,
    daily_wind_speed REAL,
    daily_snowfall REAL
);

CREATE TABLE IF NOT EXISTS taxi_trips (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pickup_datetime DATETIME,
    dropoff_datetime DATETIME,
    trip_distance REAL,
    rate_code_id INTEGER,
    pickup_location_id INTEGER,
    dropoff_location_id INTEGER,
    base_passenger_fare REAL,
    rush_hour_surcharge REAL,
    mta_tax REAL,
    tip_amount REAL,
    to

In [179]:
# create the tables with the schema files
from sqlalchemy import text

# Assuming `DATABASE_SCHEMA_FILE` contains your schema as a string
DATABASE_SCHEMA_FILE = "schema.sql"

# Read the schema file
with open(DATABASE_SCHEMA_FILE, "r") as schema_file:
    schema = schema_file.read()

# Split the schema into individual statements
statements = schema.strip().split(";")
statements = [stmt.strip() for stmt in statements if stmt.strip()]  # Remove empty statements


# Execute each statement individually
with engine.connect() as connection:
    for statement in statements:
        try:
            connection.execute(text(statement))
            print(f"Executed: {statement}")
        except Exception as e:
            print(f"Error executing statement: {statement}")
            print(f"Exception: {e}")


2024-12-02 19:24:34,122 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-02 19:24:34,133 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATETIME,
    latitude REAL,
    longitude REAL,
    hourly_precipitation REAL,
    hourly_wind_speed REAL,
    daily_snowfall REAL,
    daily_id INTEGER, 
    FOREIGN KEY (daily_id) REFERENCES daily_weather(id)
)
2024-12-02 19:24:34,138 INFO sqlalchemy.engine.Engine [generated in 0.01897s] ()
Executed: CREATE TABLE IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATETIME,
    latitude REAL,
    longitude REAL,
    hourly_precipitation REAL,
    hourly_wind_speed REAL,
    daily_snowfall REAL,
    daily_id INTEGER, 
    FOREIGN KEY (daily_id) REFERENCES daily_weather(id)
)
2024-12-02 19:24:34,142 INFO sqlalchemy.engine.Engine CREATE TABLE IF NOT EXISTS daily_weather (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date DATETIME,
    lati

In [180]:
from sqlalchemy import inspect

inspector = inspect(engine)
tables = inspector.get_table_names()
print("Available tables:", tables)


2024-12-02 19:24:38,997 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-02 19:24:39,001 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-12-02 19:24:39,004 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-02 19:24:39,012 INFO sqlalchemy.engine.Engine ROLLBACK
Available tables: ['daily_weather', 'hourly_weather', 'taxi_trips', 'uber_trips']


In [181]:
columns = inspector.get_columns('taxi_trips')
for column in columns:
    print(f"Column: {column['name']}, Type: {column['type']}")


2024-12-02 19:24:43,288 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-02 19:24:43,305 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("taxi_trips")
2024-12-02 19:24:43,357 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-02 19:24:43,370 INFO sqlalchemy.engine.Engine ROLLBACK
Column: id, Type: INTEGER
Column: pickup_datetime, Type: DATETIME
Column: dropoff_datetime, Type: DATETIME
Column: trip_distance, Type: REAL
Column: rate_code_id, Type: INTEGER
Column: pickup_location_id, Type: INTEGER
Column: dropoff_location_id, Type: INTEGER
Column: base_passenger_fare, Type: REAL
Column: rush_hour_surcharge, Type: REAL
Column: mta_tax, Type: REAL
Column: tip_amount, Type: REAL
Column: tolls_amount, Type: REAL
Column: improvement_surcharge, Type: REAL
Column: total_amount, Type: REAL
Column: congestion_surcharge, Type: REAL
Column: airport_fee, Type: REAL


### Add Data to Database

In [197]:
# final_taxi_data, uber_data, hourly_weather_data, daily_weather_data

# Load sampled trip data into each database
final_taxi_data.to_sql('taxi_trips', con=engine, if_exists='replace', index=False)
final_uber_data.to_sql('uber_trips', con=engine, if_exists='replace', index=False)
hourly_weather_data.to_sql('hourly_weather', con=engine, if_exists='replace', index=False)
daily_weather_data.to_sql('daily_weather', con=engine, if_exists='replace', index=False)


2024-12-02 19:48:46,536 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-02 19:48:46,552 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("taxi_trips")
2024-12-02 19:48:46,603 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-02 19:48:46,606 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("taxi_trips")
2024-12-02 19:48:46,607 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-02 19:48:46,611 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-12-02 19:48:46,612 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-02 19:48:46,622 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='view' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2024-12-02 19:48:46,627 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-12-02 19:48:46,630 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("taxi_trips")
2024-12-02 19:48:46,631 INFO sqlalchemy.engine.Engine [raw sql] ()
202

2024-12-02 19:48:46,746 INFO sqlalchemy.engine.Engine 
CREATE TABLE taxi_trips (
	pickup_datetime DATETIME, 
	dropoff_datetime DATETIME, 
	trip_distance FLOAT, 
	rate_code_id FLOAT, 
	pickup_location_id BIGINT, 
	dropoff_location_id BIGINT, 
	base_passenger_fare FLOAT, 
	rush_hour_surcharge FLOAT, 
	mta_tax FLOAT, 
	tip_amount FLOAT, 
	tolls_amount FLOAT, 
	improvement_surcharge FLOAT, 
	total_amount FLOAT, 
	congestion_surcharge FLOAT, 
	airport_fee FLOAT, 
	zone TEXT, 
	location_id INTEGER, 
	centroid_lat FLOAT, 
	centroid_lon FLOAT
)


2024-12-02 19:48:46,960 INFO sqlalchemy.engine.Engine [no key 0.20656s] ()
2024-12-02 19:48:49,237 INFO sqlalchemy.engine.Engine INSERT INTO taxi_trips (pickup_datetime, dropoff_datetime, trip_distance, rate_code_id, pickup_location_id, dropoff_location_id, base_passenger_fare, rush_hour_surcharge, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount, congestion_surcharge, airport_fee, zone, location_id, centroid_lat, centroid_lon) V

56098

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

### Query 1

In [ ]:
QUERY_1_FILENAME = ""

QUERY_1 = """
TODO
"""

In [ ]:
# execute query either via sqlalchemy
with engine.connect() as con:
    results = con.execute(db.text(QUERY_1)).fetchall()
results

# or via pandas
pd.read_sql(QUERY_1, con=engine)

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)

## Part 4: Visualizing the Data

### Visualization 1

In [ ]:
# use a more descriptive name for your function
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_1():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplementedError()

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)